In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pyodbc
import os
import osmnx as ox
import networkx as nx
import geonetworkx as gnx
from networkx.classes.filters import no_filter
from datetime import date

SHARED_PROJECT_PATH = '...'
SHARED_PROJECT_PATH_POLY = '...'

today = date.today().strftime('%d-%m-%Y')

In [ ]:
# Read clustered accident dataset
df_acc = gpd.read_file(os.path.join(SHARED_PROJECT_PATH, 'data', 'Waze_accidents', today, f'BP_acc-clust-airdist_{today}.json'))

In [ ]:
# Read processed OSM raod network data
gdf_graph_edges = gpd.read_file(os.path.join(SHARED_PROJECT_PATH, 'data', 'OSM_road_network', today, f'GDF_BP-network-conn-undir_{today}.json'), driver='GeoJSON').to_crs('EPSG:23700')

In [ ]:
# Add number of accidents per year to the road network - year: 01.05.2019 - 30.04.2020
# Determine year of accident alerts
df_acc = df_acc[(df_acc.ts >= '2019-05-01') & (df_acc.ts < '2024-05-01')].copy()
df_acc['year'] = 'acc_no_19-20'
for year in range(19, int(today[-2:])):
    df_acc.loc[df_acc[(df_acc.ts >= f'20{year}-05-01') & (df_acc.ts < f'20{year+1}-05-01')].index, 'year'] = f'acc_no_{year}-{year+1}'
# Join accident number data to road network dataframe
df_acc_joined = gpd.sjoin_nearest(df_acc, gdf_graph_edges, how='left', distance_col='dist', max_distance=15)
df_acc_joined_gr = df_acc_joined.groupby(['gnx_edge_id', 'year']).size().to_frame().reset_index().pivot(index='gnx_edge_id', columns='year', values=0).fillna(0)
gdf_graph_edges.set_index('gnx_edge_id', inplace=True)
gdf_graph_edges = gdf_graph_edges.join(df_acc_joined_gr, how='left')
gdf_graph_edges[gdf_graph_edges.columns[-5:]] = gdf_graph_edges[gdf_graph_edges.columns[-5:]].fillna(0)

In [ ]:
# Save road network with accident alert numbers as geojson
gdf_graph_edges.to_file(os.path.join(SHARED_PROJECT_PATH, 'data', 'OSM_road_network', today, f'BP_safety-network_{today}.json'), driver='GeoJSON')